In [1]:
!pip install pandas

In [2]:
!pip install networkx

In [3]:
!pip install joblib

In [4]:
!pip install gensim

In [38]:
from network import CNetwork

In [39]:
import pandas as pd
import numpy as np

In [40]:
from utils import verifyDir

In [41]:
name_dataset = "dataset_1"
dataset_path = "datasetsv2/"
length_cut = 10000
random_flag = True

In [42]:
auxiliar_path = 'auxiliar_folder/' + name_dataset   + '/'
verifyDir(auxiliar_path)

In [43]:
df = pd.read_csv(dataset_path + name_dataset + ".csv")

In [44]:
df.head(5)

,label,text,book
0,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Christine Gehring,...",Right Ho Jeeves
1,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Suzanne L. Shell, ...",Tales Of St Austin
2,Joseph Conrad,And that last\n\nword was the single word of ...,Victory
3,Joseph Conrad,\n\n\n\nIf I have ever had these gifts in any ...,Under Western Eyes
4,Bram Stoker,\n\n\n\n Lond...,The Lady Of The Shroud


In [45]:
from utils.text_processing import get_min_len_corpus

In [46]:
print("Min Length:", get_min_len_corpus(list(df["text"])))

Min Length: 47826


In [47]:
from utils.text_processing import get_corpus, get_random_corpus

In [48]:
texts = list(df['text'])

In [49]:
corpus, segmented_corpus = get_corpus(texts, length_cut)

In [50]:
from utils.text_processing import remove_puntuaction

In [51]:
segmented_corpus = [[[remove_puntuaction(w) for w in t] for t in p] for p in segmented_corpus ]

In [52]:
selected_corpus, words_features, word_index, index_word = get_random_corpus(segmented_corpus)

In [53]:
len(selected_corpus)

78

In [54]:
labels = list(df['label'])

In [55]:
total_classes = list(set(labels))  ## or author
print("Total classes: {}".format(len(total_classes)))
number_books = (df[df['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = {cat: index for index, cat in enumerate(total_classes)}

Total classes: 13
Total entities for each class in train: 6


In [56]:
y = [dict_categories[y] for y in labels]

In [57]:
total_classes

['Bram Stoker',
 'Joseph Conrad',
 'Pelham Grenville',
 'Mark Twain',
 'Thomas Hardy',
 'Hector Hugh',
 'Charles Dickens',
 'George Eliot',
 'Charles Darwin',
 'Daniel Defoe',
 'Jane Austen',
 'Allan Poe',
 'Arthur Conan Doyle']

In [58]:
def read_embeddings(fname):
    with open(fname) as f:
        f.readline()
        names = []
        data = []
        for line in f:
            line = line.strip().split(",")
            names.append(line[0])
            data.append([float(v) for v in line[1:]])
        return names, data

In [59]:
def organize_data(graph2label, embed_vnames, embed_vecs):
    N = len(graph2label)
    data = [0 for _ in range(N)]
    labels = [0 for _ in range(N)]
    for i in range(len(graph2label)):
        idx = embed_vnames[i]
        data[int(idx)] = embed_vecs[i]
        labels[int(idx)] = graph2label[idx]
    return np.array(data), np.array(labels)

In [60]:
from utils.igraph2json import igraph2json1
import os

In [61]:
def graph2vec1(path, operating_system, networks, labels):
    networks_dir = path + 'dataset/'
    emb_file = path + 'features/nci1.csv'
    graph2label = {}
    try: 
        os.mkdir(in_network)
        os.mkdir(path + 'features/')
    except:
        print("Existe")
    for (i, netw), label in zip(enumerate(networks), labels):
        graph2label[str(i)] = label
        igraph2json1(netw, networks_dir + str(i) + ".json")
    if operating_system == 'linux':
        path_command = 'python ./graph2vec/src/graph2vec.py --input-path ' + networks_dir + ' --output-path ' + emb_file + ' --dimensions ' + str(512)
    os.system(path_command)
    embed_vnames, embed_vecs = read_embeddings(emb_file)
    X, y = organize_data(graph2label, embed_vnames, embed_vecs)
    return X, y


In [62]:
def graph2vec(path, operating_system, networks):
    in_network = path + 'dataset/'
    extra_file = path + 'features/nci1.csv'
    try: 
        os.mkdir(in_network)
        os.mkdir(path + 'features/')
    except:
        print("Existe")
    for i, netw in enumerate(networks):
        igraph2json1(netw, in_network + str(i) + ".json")
    if operating_system == 'linux':
        path_command = 'python graph2vec/src/graph2vec.py --input-path ' + in_network + ' --output-path ' + extra_file + ' --dimensions ' + str(512)
    os.system(path_command)
    g2v = pd.read_csv(extra_file, sep=',', lineterminator='\n')
    g2v.set_index("type")
    all_network_features = [g2v.loc[i].values for i, _ in enumerate(networks)]
    print("graph2vec")
    return all_network_features

In [63]:
def get_graph2vec_features(sequences):
    network_features = pd.DataFrame()
    nets = []
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=index_word, percentages=None, path=auxiliar_path)
        network = obj.create_network()
        nets.append(network)
    X = np.array(graph2vec("", "linux", nets))
    return X

In [64]:
X = get_graph2vec_features(selected_corpus)

Nodes: 2455 - Edges: 7350
Nodes: 2330 - Edges: 7408
Nodes: 2589 - Edges: 7687
Nodes: 2684 - Edges: 7776
Nodes: 2341 - Edges: 7370
Nodes: 1734 - Edges: 6752
Nodes: 2125 - Edges: 6897
Nodes: 2255 - Edges: 7316
Nodes: 2090 - Edges: 6992
Nodes: 2520 - Edges: 7471
Nodes: 2419 - Edges: 7407
Nodes: 2272 - Edges: 7153
Nodes: 2890 - Edges: 7855
Nodes: 1667 - Edges: 6489
Nodes: 2678 - Edges: 7700
Nodes: 1797 - Edges: 6530
Nodes: 2975 - Edges: 7811
Nodes: 2846 - Edges: 7814
Nodes: 2492 - Edges: 7652
Nodes: 2433 - Edges: 7644
Nodes: 2193 - Edges: 4858
Nodes: 2231 - Edges: 7180
Nodes: 2185 - Edges: 7378
Nodes: 2893 - Edges: 7913
Nodes: 1540 - Edges: 5405
Nodes: 2560 - Edges: 7662
Nodes: 2453 - Edges: 7451
Nodes: 2741 - Edges: 7739
Nodes: 2413 - Edges: 7685
Nodes: 2634 - Edges: 7536
Nodes: 1937 - Edges: 6480
Nodes: 2906 - Edges: 7803
Nodes: 2750 - Edges: 7512
Nodes: 2287 - Edges: 7522
Nodes: 1646 - Edges: 6209
Nodes: 1973 - Edges: 7126
Nodes: 2080 - Edges: 7143
Nodes: 2806 - Edges: 7747
Nodes: 2978 

  0%|          | 0/78 [00:00<?, ?it/s]


Feature extraction started.



100%|██████████| 78/78 [00:01<00:00, 72.36it/s]


Optimization started.



graph2vec


In [65]:
print("Lenght of features:", X.shape)

Lenght of features: (78, 513)


# Training with SVM

In [66]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [67]:
RANDOM_SEED = 20
FRAC_TRAIN = 0.6

In [68]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=(1.0 - FRAC_TRAIN), random_state=RANDOM_SEED)

In [69]:
from sklearn.svm import SVC

In [70]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(y_val)
print(predicted)
#print accuracy_score
print("Accuracy : " + str(accuracy_score(y_val, predicted)))

print("Micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[11, 4, 5, 9, 12, 12, 3, 5, 1, 11, 3, 11, 8, 1, 9, 8, 6, 4, 10, 2, 12, 0, 2, 4, 3, 7, 7, 10, 0, 9, 10, 6]
[ 8  2  8  8  8  2  8  6  9  8 12  6  1  9  8  8  8 10  8  8  8  7  9  8
  5 10 10  8  0  8  8  7]
Accuracy : 0.0625
Micro f-measure 0.0625
